<a href="https://colab.research.google.com/github/Mechanics-Mechatronics-and-Robotics/CV-2025/blob/main/Lab_1_Feature_Extraction_and_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab. \#1. Feature Extraction and Machine Learning

## Problem Statement

The lab deals with comparison of two approaches to machine learning (ML) and computer vision (CV). The first approach is processing of hand-designed features, e.g. geometric features of objects in images, with an ML classification model. The second approach is using of the ML model for both, the automatic feature extraction and the following classification.

The MNIST database of handwritten digits has a training set of 60,000 examples, and a test set of 10,000 examples.

The hand-designed features can be extracted with standart tools in [scikit-learn](https://scikit-learn.org/1.5/modules/feature_extraction.html)

## Tasks and Requirements

Bonus
* apply a t-SNE model to visualize both, the original images dataset, and hand-extracted features of the dataset

# Import and Install Libraries

In [1]:
!pip install pytorch-lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 37.4 MB/s eta 0:00:00


In [20]:
#Pytorch modules
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
from torchvision import datasets, transforms

#Numpy
import numpy as np

#Pandas
import pandas as pd

#Lightning & logging
import pytorch_lightning as pl

#Data observation
import os
from pathlib import Path

#Plotting
import matplotlib.pyplot as plt
import seaborn as sns


# Set the Model

## Simulation Settings

Check the current directory

In [4]:
os.getcwd() #returns the current working directory

'/content'

Paths and initializations of the weights

In [13]:
# Path to the folder where the dataset is saved
DATASET_PATH = os.environ.get("PATH_DATASET", "data/")
print(f'DATASET_PATH: {DATASET_PATH}')

# Path to the folder where the trained or pretrained models are saved
CHECKPOINT_PATH = os.environ.get("PATH_CHECKPOINT", "saved_models")
print(f'CHECKPOINT_PATH: {CHECKPOINT_PATH}')

os.makedirs(DATASET_PATH, exist_ok=True) #create the directory
os.makedirs(CHECKPOINT_PATH, exist_ok=True) #create the directory

isINFERENCE = False # inference mode with downloading the saved weights
isPretrained = False # use the pretrained weights when training

# Function for setting the seed to implement parallel tests
seed = 42 # random seeds are 42, 0, 17, 9, 3
pl.seed_everything(seed)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

INFO:lightning_fabric.utilities.seed:Seed set to 42


DATASET_PATH: data/
CHECKPOINT_PATH: saved_models


## Logging

## Dataset

Summary

In [21]:
DATASET = 'MNIST'
ns = {'train': 50000, 'val': 10000, 'test': 10000}

SIZE = 28 #image size
NUM_CLASSES = 10
CLASS_NAMES = ['zero' ,'one', 'two', 'three', 'four',
               'five', 'six', 'seven', 'eight', 'nine']

Normalization parameters

In [15]:
mean = np.array([0.1307])
std  = np.array([0.3081])

Transforms

In [16]:
data_transforms = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize(mean, std)])

## Collect hyperparameters

In [17]:
#Model parameters
LOSS_FUN = 'CE'
ARCHITECTURE = 'MLP'
lr = 0.0001 #
n = 100 # number of epochs

batch_size = 32
num_workers = 8

optimization_algorithm = 'Adam'# 'SGD','Adam'
MOMENTUM = 0.9
WD = 1e-1 # weight decay

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
roundRun = 5 #number of digits in the results performance

#Visualization
figSize = (8,8)
nSamples = 4
numBins = 50

#Summary: hyperparameters
hyperparameters = {
    "seed": seed,
    "lr": lr,
    "isINFERENCE": isINFERENCE,
    "isPretrained": isPretrained,
    "dataset": DATASET,
    "n_classes": NUM_CLASSES,
    "class_names": CLASS_NAMES,
    "bs": batch_size,
    "num_workers": num_workers,
    "num_epochs": n,
    "model_filename": ARCHITECTURE,
    "optimization_algorithm": optimization_algorithm,
    "momentum": MOMENTUM,
    "criterion": LOSS_FUN,
    "weight decay": WD,
    "device": DEVICE,
    "fig_size": figSize,
    "num_samples": nSamples,
    "num_bins": numBins,
}

# Functions

## Lightning

Training module

Callbacks

## Model

MLP

In [22]:
class MLP(pl.LightningModule):

  def __init__(self):
    super(MLP, self).__init__()

    # mnist images are (1, 28, 28) (channels, width, height)
    self.layer_1 = torch.nn.Linear(SIZE * SIZE, 128)
    self.layer_2 = torch.nn.Linear(128, 256)
    self.layer_3 = torch.nn.Linear(256, NUM_CLASSES)

  def forward(self, x):
    batch_size, channels, width, height = x.siz()

    # (b, 1, SIZE, SIZE) -> (b, 1*SIZE*SIZE)
    x = x.view(batch_size, -1)

    # layer 1
    x = self.layer_1(x)
    x = torch.relu(x)

    # layer 2
    x = self.layer_2(x)
    x = torch.relu(x)

    # layer 3
    x = self.layer_3(x)

    # # probability distribution over labels
    # x = torch.log_softmax(x, dim=1)

    return x

## Loss

In [19]:
# Cross entropy loss
class CEloss(nn.Module):
    def __init__(self):
        super(CEloss, self).__init__()

    def forward(self,x,y):
        prob = nn.functional.softmax(x,1)
        log_prob = -1.0 * torch.log(prob)
        loss = log_prob.gather(1, y.unsqueeze(1))
        loss = loss.mean()
        return loss

## Visualization

In [ ]:
def imshow(inp, title):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    inp = ((std * inp) + mean)
    inp = np.clip(inp, 0, 1)
    #plt.grid(visible=None, which='major',axis='both')
    plt.axis('off')
    plt.imshow(inp)
    plt.title(title)
    plt.show()

In [ ]:
def hist(df, values, histSize):
    sns.set(style="darkgrid")
    sns.set(rc={'figure.figsize': histSize})

    n = hyperparameters['num_bins']

    sns.histplot(data=df[df['True or false prediction'] == True], x = values, color="skyblue",
                 label='True predictions',  bins=n, kde=True)
    sns.histplot(data=df[df['True or false prediction'] == False],x = values, color="red",
                 label='False predictions', bins=n, kde=True)
    plt.legend()
    return